# Part 1: Visual Document Search with ColQwen and Elasticsearch `rank_vectors`

This notebook demonstrates how to reproduce and enhance the concepts from the Elastic Search Labs blog post '[Searching complex documents with ColPali](https://www.elastic.co/search-labs/blog/elastiacsearch-colpali-document-search)'.

**Key Enhancements:**

1.  **Model Upgrade**: We use the **`TomoroAI/tomoro-colqwen3-embed-4b`** model, a top-performing model on the ViDoRe benchmark.
2.  **Real-world Dataset**: We use sample images from the **RVL-CDIP** dataset to demonstrate capabilities on complex, real-world documents.
3.  **Robust Connection Logic**: The connection logic automatically detects whether the environment variable specifies a **Cloud ID** or a **Host URL**, ensuring a stable connection.

### Step 0: Setup Environment

**[EN]** Install the necessary libraries.<br>
**[KR]** 필요한 라이브러리를 설치합니다.

In [2]:
!pip install --upgrade pip jupyter jupyterlab ipywidgets tqdm
!pip install -q "git+https://github.com/illuin-tech/colpali.git"
!pip install -q "transformers>=4.41.0" accelerate Pillow elasticsearch python-dotenv tqdm


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [ ]:
!pip install wheel setuptools
!MAX_JOBS=4 pip install flash-attn --no-build-isolation

  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Using cached flash_attn-2.8.3.tar.gz (8.4 MB)
  Preparing metadata (pyproject.toml) ... done
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
Using cached einops-0.8.1-py3-none-any.whl (64 kB)
ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "/home/rag/Studies/workshop/colpali/.venv/bin/pip", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/rag/Studies/workshop/colpali/.venv/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 80, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rag/Studies/workshop/colpali/.venv/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 158, in main
    with self.main_context():
  File "/home/rag/.pyenv/versions/3.11.13/lib/python3.11/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/home/rag/

### Step 1: Import Libraries and Set Environment Variables

**[EN]** Import necessary libraries and load environment variables from the `elastic.env` file.<br>
**[KR]** 필요한 라이브러리를 임포트하고, `elastic.env` 파일에서 환경 변수를 로드합니다.

In [ ]:
import torch
import os
import glob
import base64
from io import BytesIO
from tqdm.notebook import tqdm
from PIL import Image
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
from IPython.display import display, HTML

dotenv_path = 'elastic.env'
load_dotenv(dotenv_path=dotenv_path)

ES_URL = os.getenv("ES_URL")
ES_API_KEY = os.getenv("ES_API_KEY")

if not ES_URL or not ES_API_KEY:
    raise ValueError(f"Please ensure '{dotenv_path}' contains ES_URL and ES_API_KEY.")

INDEX_NAME = "colqwen3-rvlcdip-demo-part1"
VECTOR_FIELD_NAME = "colqwen_vectors"
SAMPLED_DATA_DIR = "samples"

### Step 2: Load the ColQwen Model

**[EN]** Load the `tsystems/colqwen2.5-3b-multilingual-v1.0` model and its processor.<br>
**[KR]** `tsystems/colqwen2.5-3b-multilingual-v1.0` 모델과 프로세서를 로드합니다.

In [ ]:
from colpali_engine.models import ColQwen3, ColQwen3Processor
from transformers import AutoConfig

device_map = "cpu"
if torch.backends.mps.is_available():
    device_map = "mps"
elif torch.cuda.is_available():
    device_map = "cuda:0"
print(f"Using device: {device_map}")

MODEL_NAME = "TomoroAI/tomoro-colqwen3-embed-4b"

# Load config first and fix rope_scaling if None
config = AutoConfig.from_pretrained(MODEL_NAME, trust_remote_code=True)
if hasattr(config, 'text_config') and config.text_config.rope_scaling is None:
    config.text_config.rope_scaling = {"mrope_section": [24, 20, 20], "type": "default"}

# Try flash_attention_2 first, fall back to sdpa if not available
attn_impl = "flash_attention_2" if device_map != "cpu" else "eager"
try:
    import flash_attn
except ImportError:
    attn_impl = "sdpa" if device_map != "cpu" else "eager"
    print(f"flash-attn not installed, using '{attn_impl}' attention instead.")

model = ColQwen3.from_pretrained(
    MODEL_NAME,
    config=config,
    torch_dtype=torch.bfloat16 if device_map != "cpu" else torch.float32,
    device_map=device_map,
    attn_implementation=attn_impl,
    trust_remote_code=True
).eval()
processor = ColQwen3Processor.from_pretrained(MODEL_NAME, trust_remote_code=True)
print(f"Model '{MODEL_NAME}' loaded successfully on device '{model.device}'.")

### Step 3: Load RVL-CDIP Sample Data

**[EN]** Load the list of RVL-CDIP sample image files from the `samples` directory.<br>
**[KR]** `samples` 디렉터리에서 RVL-CDIP 샘플 이미지 파일 목록을 가져옵니다.

In [ ]:
if not os.path.exists(SAMPLED_DATA_DIR):
    raise FileNotFoundError(f"Sample data directory not found at '{SAMPLED_DATA_DIR}'. Please create it in the same directory as this notebook.")

image_paths = []
for category_dir in os.listdir(SAMPLED_DATA_DIR):
    full_category_path = os.path.join(SAMPLED_DATA_DIR, category_dir)
    if os.path.isdir(full_category_path):
        image_paths.extend(glob.glob(os.path.join(full_category_path, '*.*')))
print(f"Found {len(image_paths)} sample images.")

### Step 4: Define Embedding Helper Functions

**[EN]** Define helper functions to generate multi-vector embeddings for document images and text queries.<br>
**[KR]** 문서 이미지와 텍스트 질의의 다중 벡터 임베딩을 생성하는 함수를 정의합니다.

In [ ]:
def create_colqwen_document_vectors(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor.process_images([image]).to(model.device)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.cpu().to(torch.float32).numpy().tolist()[0]
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def create_colqwen_query_vectors(query_text):
    inputs = processor.process_queries([query_text]).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.cpu().to(torch.float32).numpy().tolist()[0]

### Step 5 & Step 6 : Connect to Elasticsearch and Create an Index and Index the Data

**[EN]** Connect to Elasticsearch and create an index with a `rank_vectors` field and index the loaded sample images into Elasticsearch. <br>
**[KR]** Elasticsearch에 연결하고 `rank_vectors` 필드를 포함하는 인덱스를 생성한 후 로드된 샘플 이미지들을 Elasticsearch에 인덱싱합니다.

In [ ]:
# This combined cell handles both index creation and document indexing
# to ensure data integrity based on the document count.

# --- Logic from original Step 5: Connect to Elasticsearch and Manage Index ---

# Automatically detect if ES_URL is a Cloud ID or a Host URL.
if ':' in ES_URL and not ES_URL.startswith('http'):
    # Connect using Cloud ID
    print("Connecting using Cloud ID...")
    es = Elasticsearch(
        cloud_id=ES_URL,
        api_key=ES_API_KEY,
        request_timeout=20
    )
else:
    # Connect using Host URL
    print("Connecting using Host URL...")
    es = Elasticsearch(
        hosts=[ES_URL],
        api_key=ES_API_KEY,
        request_timeout=20,
        verify_certs=False,  # Disable SSL certificate verification for self-signed certs
        ssl_show_warn=False  # Suppress SSL warnings
    )

print(f"Connected to Elasticsearch version: {es.info()['version']['number']}")

# Define the index mapping
mapping = {
    "properties": {
        VECTOR_FIELD_NAME: {"type": "rank_vectors", "dims": 320},
        "image_path": {"type": "keyword"},
        "category": {"type": "keyword"}
    }
}

# --- Logic from original Step 6: Validate and Conditionally Index Documents ---

# Default to an invalid state to trigger re-indexing if checks fail.
is_index_valid = False

if es.indices.exists(index=INDEX_NAME):
    try:
        doc_count = es.count(index=INDEX_NAME)['count']
        print(f"Index '{INDEX_NAME}' already exists with {doc_count} documents.")
        if doc_count == 1600:
            is_index_valid = True
    except Exception as e:
        print(f"Could not retrieve document count for existing index. Will recreate. Error: {e}")
else:
    print(f"Index '{INDEX_NAME}' not found.")

# If the index is not valid (doesn't exist or has the wrong doc count),
# delete it, recreate it, and re-index all documents.
if not is_index_valid:
    print(f"Index state is not valid. Starting complete re-indexing process...")
    
    # 1. Delete the index if it exists, to ensure a clean state.
    if es.indices.exists(index=INDEX_NAME):
        print(f"Deleting existing index '{INDEX_NAME}'...")
        es.indices.delete(index=INDEX_NAME)
    
    # 2. Create a new, empty index with the correct mapping.
    print(f"Creating new index '{INDEX_NAME}'...")
    es.indices.create(index=INDEX_NAME, mappings=mapping)
    print(f"Index created successfully.")

    # 3. Index all 1600 documents.
    MAX_DOCS_TO_INDEX = 1600
    docs_to_index = image_paths[:min(len(image_paths), MAX_DOCS_TO_INDEX)]
    print(f"Indexing {len(docs_to_index)} documents...")

    for path in tqdm(docs_to_index, desc="Indexing Documents"):
        doc_id = os.path.splitext(os.path.basename(path))[0]
        category = os.path.basename(os.path.dirname(path))
        vectors = create_colqwen_document_vectors(path)
        
        if vectors:
            es_doc = {
                VECTOR_FIELD_NAME: vectors,
                "image_path": path,
                "category": category
            }
            es.index(index=INDEX_NAME, id=doc_id, document=es_doc)

    es.indices.refresh(index=INDEX_NAME)
    print("\nIndexing complete.")
else:
    print(f"Index '{INDEX_NAME}' is valid with 1600 documents. Skipping all steps.")

### Step 7: Search and Visualize Results

**[EN]** Perform a search using `script_score` and new `maxSimDotProduct` function to calculate the similarity between our query and the image vectors in Elasticsearch and visualize the results.<br>
**[KR]** `script_score`와 새로 출시된 `maxSimDotProduct`를 사용하여 쿼리와 이미지 벡터 간의 유사성을 계산하여 검색을 수행하고  결과를 시각화합니다.

In [ ]:
# Add necessary libraries
import json
import re
import time

def remove_query_vector_from_explanation(explanation_obj):
    """
    Recursively traverses the explanation object to remove the lengthy 'query_vector' parameter
    from both structured 'params' and string-based 'description' fields.
    """
    if isinstance(explanation_obj, dict):
        if 'description' in explanation_obj and isinstance(explanation_obj['description'], str):
            explanation_obj['description'] = re.sub(
                r"query_vector=\[\[.*?\]\]", 
                "query_vector=[...vector omitted for brevity...]", 
                explanation_obj['description']
            )
        if 'params' in explanation_obj and 'query_vector' in explanation_obj['params']:
            explanation_obj['params']['query_vector'] = "[...vector omitted for brevity...]"
        for key, value in explanation_obj.items():
            remove_query_vector_from_explanation(value)
    elif isinstance(explanation_obj, list):
        for item in explanation_obj:
            remove_query_vector_from_explanation(item)
    return explanation_obj

def search_and_display_with_explain(query, es_client):
    print(f"\nSearching for: '{query}'")
    query_vectors = create_colqwen_query_vectors(query)
    
    # This is the original, full "late interaction" search query.
    es_query = {
        "_source": ["image_path", "category"],
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": f"maxSimDotProduct(params.query_vector, '{VECTOR_FIELD_NAME}')",
                    "params": {"query_vector": query_vectors},
                },
            }
        },
        "size": 5,
        "explain": True  # Enable the Explain API
    }
    
    # --- Measure Search Latency ---
    start_time = time.time()
    results = es_client.search(index=INDEX_NAME, body=es_query)
    end_time = time.time()
    latency_ms = (end_time - start_time) * 1000
    print(f"🚀 Search Latency: {latency_ms:.2f} ms")
    # --- End of Measurement ---

    hits = results["hits"]["hits"]
    
    # --- Display visual results ---
    html = "<table><tr>"
    for hit in hits:
        doc_id = hit["_id"]
        score = hit["_score"]
        path = hit["_source"]["image_path"]
        category = hit["_source"]["category"]
        try:
            with open(path, "rb") as image_file:
                img_str = base64.b64encode(image_file.read()).decode()
                html += f"""
                <td style='text-align: center; vertical-align: top; padding: 10px; border: 1px solid #ddd;'>
                    <img src='data:image/png;base64,{img_str}' width='200'><br>
                    <b>ID:</b> {doc_id}<br>
                    <b>Score:</b> {score:.4f}<br>
                    <b>Category:</b> {category}
                </td>
                """
        except Exception as e:
            print(f"Could not display image {path}: {e}")
    html += "</tr></table>"
    
    if hits:
        display(HTML(html))
        
        # --- Display the cleaned explanation for the top result ---
        top_hit_explanation = hits[0].get("_explanation")
        if top_hit_explanation:
            print("\n--- Explanation for Top Result (Rank #1) ---")
            cleaned_explanation = remove_query_vector_from_explanation(top_hit_explanation)
            print(json.dumps(cleaned_explanation, indent=2))
            
    else:
        print("No results found.")

# Perform a series of searches to demonstrate the model's capabilities with explanations.
search_and_display_with_explain("Do you have a benefits policy change notice from HR?", es)
search_and_display_with_explain("인사팀에서 보내온 복리후생 정책 변경 안내문이 있나?", es)

### Step 8: Free up Memory

**[EN]** This cell releases PyTorch's cached memory. However, to **completely** free GPU memory in Jupyter, you must **restart the kernel** (`Kernel > Restart Kernel`).

**[KR]** 이 셀은 PyTorch의 캐시된 메모리를 해제합니다. 그러나 Jupyter에서 GPU 메모리를 **완전히** 해제하려면 **커널을 재시작**해야 합니다 (`Kernel > Restart Kernel`).

In [ ]:
import gc

try:
    if 'model' in globals() and model is not None:
        model.to('cpu')
        del model
    if 'processor' in globals() and processor is not None:
        del processor
except:
    pass

gc.collect()

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
elif torch.backends.mps.is_available():
    torch.mps.empty_cache()

print("✅ 메모리 정리 완료!")

# Jupyter Notebook에서만 작동 (VS Code에서는 작동 안 함)
from IPython.display import display, Javascript
display(Javascript('Jupyter.notebook.kernel.restart()'))

print("💡 GPU 메모리를 완전히 해제하려면: Kernel → Restart Kernel")